In [78]:
# import cartopy.crs as ccrs
# import easygems.healpix as egh
# import intake
# import matplotlib.pyplot as plt
# import numpy as np

import xdggs
import xarray as xr
import numpy as np
import dask.array as dsa
# zoom = 15
zoom = 10
path = f"https://nowake.nicam.jp/files/220m/data_healpix_{zoom}.zarr"
ds = xr.open_dataset(path, engine='zarr', chunks={})
ds = ds.rename({'cell':'cell_ids'})

# cell_ids = xr.DataArray(dsa.arange(ds.cell_ids.size,chunks=(1048576)),dims=['cell_ids'], name='cell_ids')
# ds = ds.assign_coords({'cell_ids': cell_ids})

# the coordinate labels are wayyy bigger than my memory! So xarray loading them by default is a non-starter.
# Trying https://github.com/pydata/xarray/issues/1650#issuecomment-1697282386
cell_ids = xr.Coordinates({"cell_ids": ("cell_ids", dsa.arange(ds.cell_ids.size, chunks=(1048576)))}, indexes={})
ds = ds.assign_coords(cell_ids)

ds.cell_ids.attrs = {
    "grid_name": "healpix",
    "level": zoom,
    "indexing_scheme": "nested",
}
ds

<xarray.Dataset> Size: 604MB
Dimensions:       (time: 1, cell_ids: 12582912)
Coordinates:
    cell_ids      (cell_ids) int64 101MB dask.array<chunksize=(1048576,), meta=np.ndarray>
Dimensions without coordinates: time
Data variables:
    sa_cldi       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_cldw       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_lwu_toa    (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_slp_ecmwf  (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_tppn       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_vap_atm    (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ss_t2m        (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ss_tppn       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ss_u10m       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ss_v10m       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>

In [79]:
ds_decoded = xdggs.decode(ds)
ds_decoded = ds_decoded.dggs.assign_latlon_coords()
ds_decoded

<xarray.Dataset> Size: 805MB
Dimensions:       (time: 1, cell_ids: 12582912)
Coordinates:
  * cell_ids      (cell_ids) int64 101MB 0 1 2 3 ... 12582909 12582910 12582911
    latitude      (cell_ids) float64 101MB 0.0373 0.0746 ... -0.0746 -0.0373
    longitude     (cell_ids) float64 101MB 45.0 45.04 44.96 ... 315.0 315.0
Dimensions without coordinates: time
Data variables:
    sa_cldi       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_cldw       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_lwu_toa    (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_slp_ecmwf  (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_tppn       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    sa_vap_atm    (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ss_t2m        (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ss_tppn       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ss_u10m       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
    ss_v10m       (time, cell_ids) float32 50MB dask.array<chunksize=(1, 1048576), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(level=10, indexing_scheme=nested)

In [80]:
# ds_decoded = ds_decoded.dggs.sel_latlon(slice(-20, 30), slice(-20, 30))

In [82]:
test = ds_decoded['ss_u10m'].load().dggs.explore()

In [83]:
test